In [ ]:
import numpy as np 

import pandas as pd
import pandas_profiling as pp

import matplotlib.pyplot as plt
import seaborn as sns

from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import os
from IPython.display import Image

1. Прочитати опис та викачати дані з Kaggle змагання https://www.kaggle.com/c/home-credit-default-risk/data. Опис колонок знаходиться у файлі HomeCredit_columns_description.csv.
2. Для лабораторної роботи достатньо працювати лише з таблицями “application_{train|test}.csv”. З’єднувати інші таблиці не потрібно.

In [ ]:
df_description = pd.read_csv(os.path.join('/kaggle/input/home-credit-default-risk/','HomeCredit_columns_description.csv'))
df_test = pd.read_csv(os.path.join('/kaggle/input/home-credit-default-risk/','application_test.csv'))
df_train = pd.read_csv(os.path.join('/kaggle/input/home-credit-default-risk/','application_train.csv'))
df = df_train.append(df_test)
print('Опис колонок:')
df_description.head()

3. Провести EDA на таблицях “application_{train|test}.csv”: про що ці дані, які розподіли колонок, скільки missing values, наскільки збалансовані дані, яким чином обробити missing values, чим відрізняються train/test, ...?

In [ ]:
# пошук дублікатів

df_train[df_train.duplicated()]

In [ ]:
df_test[df_test.duplicated()]

In [ ]:
# інформація про сирі дані

print('Тренувальні:')
df_train.info()
df_train.describe()

In [ ]:
print('Тестові:')
df_test.info()
df_test.describe()

In [ ]:
print('Кореляція з TARGET:')
df_train.corr()['TARGET'].sort_values()

In [ ]:
# обираємо потрібні колонки

features = ['TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
    'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_GOODS_PRICE', 'NAME_INCOME_TYPE',
    'NAME_EDUCATION_TYPE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'CNT_FAM_MEMBERS', 'EXT_SOURCE_2',
    'EXT_SOURCE_3']
df_train = df_train[features]
features.remove("TARGET")
df_test = df_test[features]

In [ ]:
# шукаємо пропущені значення

print(df_train.columns[df_train.isna().any()].tolist())
print(df_test.columns[df_test.isnull().any()].tolist())

In [ ]:
# заповнюємо пропущені значення середнім

df_train[['AMT_GOODS_PRICE', 'CNT_FAM_MEMBERS', 'EXT_SOURCE_2', 'EXT_SOURCE_3']] = df_train[['AMT_GOODS_PRICE', 'CNT_FAM_MEMBERS', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].fillna(value=df_train[['AMT_GOODS_PRICE', 'CNT_FAM_MEMBERS', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean())
df_test[['EXT_SOURCE_2', 'EXT_SOURCE_3']] = df_test[['EXT_SOURCE_2', 'EXT_SOURCE_3']].fillna(value=df_test[['EXT_SOURCE_2', 'EXT_SOURCE_3']].mean())

In [ ]:
# інформація про очищені дані

print('Тренувальні:')
df_train.info()
df_train.describe()

In [ ]:
print('Тестові:')
df_test.info()
df_test.describe()

In [ ]:
# переводимо DAYS_BIRTH в додатні значення

df_train['DAYS_BIRTH'] = abs(df_train['DAYS_BIRTH'])
df_test['DAYS_BIRTH'] = abs(df_test['DAYS_BIRTH'])

In [ ]:
# кодування категоріальних даних

for c in df_train.select_dtypes(include=[object]).columns.tolist(): 
    df_train[c] = df_train[c].astype('category').cat.codes

for c in df_test.select_dtypes(include=[object]).columns.tolist(): 
    df_test[c] = df_test[c].astype('category').cat.codes

df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

In [ ]:
# проведення  кореляційного  та  причинно-наслідкового  аналізів 
print('Тренувальні:')

df_train.corr()

In [ ]:
sns.pairplot(df_train)

In [ ]:
print('Тестові:')

df_test.corr()

In [ ]:
sns.pairplot(df_test)

In [ ]:
# розподіли даних

print('Тренувальні:')
df_train.hist(figsize=(16, 20), bins = 30)

In [ ]:
print('Тестові:')
df_test.hist(figsize=(16, 20), bins = 30)

4. Побудувати класифікатор на основі проаналізованих даних. Дозволяється використати будь-який з відомих Вам методів (наприклад за допомогою https://scikit-learn.org/stable/supervised_learning.html). Для оцінки якості натренованої моделі використати train-val-split, тобто відкласти частину “application_train.csv” для валідації. Метрика - ROC_AUC.
5. Класифікувати дані з файла application_test.csv, записати результат в submission файл за прикладом “sample_submission.csv”. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train.drop('TARGET', axis='columns', inplace=False), df_train["TARGET"], test_size=0.2)

df_test_1 = pd.read_csv(os.path.join('/kaggle/input/home-credit-default-risk/','application_test.csv'))

clf = LGBMClassifier()
clf.fit(X_train, y_train)
train_preds = clf.predict(X_train)
print(f"Train Accuracy: {accuracy_score(y_train, train_preds)}")
test_preds = clf.predict(X_test)
print(f"Test Accuracy: {accuracy_score(y_test, test_preds)}")


probabilities = clf.predict_proba(df_test)
pd.DataFrame({
    'SK_ID_CURR': df_test_1['SK_ID_CURR'],
    'TARGET':     [ row[1] for row in probabilities]
}).to_csv('./sample_submission.csv',index=False)
test_preds = clf.predict_proba(X_test)[:, 1]
train_preds = clf.predict_proba(X_train)[:, 1]
lr_auc = roc_auc_score(y_test, test_preds)
tr_auc = roc_auc_score(y_train, train_preds)
print('train:ROC AUC=%.3f' % (tr_auc))
print('test:ROC AUC=%.3f' % (lr_auc))
tr_fpr, tr_tpr, _ = roc_curve(y_train, train_preds)

lr_fpr, lr_tpr, _ = roc_curve(y_test, test_preds)
plt.plot(lr_fpr, lr_tpr, marker='.', label='test')
plt.plot(tr_fpr, tr_tpr, marker='.', label='train')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

6. Залити submission файл на платформу в “Leaderboard”->“Late Submission”, отримати результат - оцінку, зробити знімок екрану і додати його до фінального ноутбука.

In [ ]:
Image(filename='/kaggle/input/resultpng/Screenshot 2021-04-27 at 18.47.16.png')